In [1]:
%%capture --no-display
from dask.distributed import Client
client = Client(address = 'tcp://192.168.62.207:8786')
client

Client Scheduler: tcp://192.168.62.207:8786 Dashboard: http://192.168.62.207:8787/status,Cluster Workers: 10 Cores: 20 Memory: 673.47 GB


In [2]:
%%time
import dask.bag as db
from takco.util import robust_json_loads_lines
fnames = 'hdfs://bricks07:9000/user/kruit/wikidump-tables/0*.jsonl'
fnames = 'hdfs://bricks07:9000/user/kruit/wikidump-tables-25pct/*.jsonl'
bag = db.read_text(fnames).map_partitions(robust_json_loads_lines).persist()
bag.count().compute()

CPU times: user 6.91 s, sys: 536 ms, total: 7.44 s
Wall time: 1min 30s


3381924